<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RecSys/blob/main/Zemlyakov_Serj/Int_1_%D0%9F%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3_2_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Интеграция модели для предсказания

In [1]:
# Работа с массивами данных
import numpy as np
from tensorflow.keras.models import load_model

# Регулярные выражения
import re

#Для работы с файлами 
import os 

# Модуль для работы с zip-архивами
from zipfile import ZipFile as Zip             

# Работа с табличными данными
import pandas as pd

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Токенизатор для преобразования текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Масштабирование данных
from sklearn.preprocessing import StandardScaler

!pip install tensorflow-addons

import tensorflow_addons as tfa

from tensorflow_addons.metrics import F1Score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 9.5 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
# Задаем константы
# Количество классов
CLASS_COUNT = 3

# Задаем словарь классов
CLASS_DICT ={'Негоден':0, 'ОграниченноГоден':1, 'Годен':2}

# Задаем словарь групп здоровья
GZ_DICT ={'':0, 'I':1, 'II':2, 'IIIа':3, 'IIIб':4}

# Функция извлечения данных о вредных факторах

def extract_harm_text(df_harm):
    result = []
    COL_HARM   = df_harm.columns.get_loc('Вредность')
    # Для всех строк таблицы: собрать значения столбцов вредности
    for row in df_harm.values:
                    result.append(str(row[COL_HARM]).split(','))
    # Возврат в виде массива
    return result

# Функция извлечения данных о Диагнозе

def extract_Diag_text(df_diag):
    result = []
    COL_DIAG    = df_diag.columns.get_loc('Диагнозы')
    # Для всех строк таблицы: собрать значения сводного Диагноза
    for row in df_diag.values:
                    result.append(str(row[COL_DIAG]).split(','))
    # Возврат в виде массива
    return result

# Функция извлечения данных о Рекомендациях

def extract_recom_text(df_diag):
    result = []
    COL_RECOM    = df_diag.columns.get_loc('Рекомендации')
    # Для всех строк таблицы: собрать значения сводного Диагноза
    for row in df_diag.values:
                    result.append(str(row[COL_RECOM]).split(','))
    # Возврат в виде массива
    return result

# Функция извлечения данных о Группе здоровья OHE

def extract_gz(df):
  GZ_COUNT = 5
  gz_list_key = list(df.ГруппаЗдоровья.values)
  gz_list = []
  for GZ in gz_list_key:
    gz_list.append(GZ_DICT[GZ])
  
  gz_data = np.array(gz_list)            # Перевод общего списка меток класса в numpy-массив
  gz_res = utils.to_categorical(gz_data, GZ_COUNT)
  return gz_res

# Функция перевода классов в OHE
def Y_to_OHE(df):
  y_list_key = list(df.ЗаключениеМК.values)
  # print (y_list_key)
  y_list = []
  for CD in y_list_key:
    y_list.append(CLASS_DICT[CD])
  # print (y_list)

  y_data = np.array(y_list)            # Перевод общего списка меток класса в numpy-массив

  y_res = utils.to_categorical(y_data, CLASS_COUNT)
  return y_data, y_res

# Преобразование текстовых данных в числовые/векторные для обучения нейросетью
# Используется встроенный в Keras токенизатор для разбиения текста и построения частотного словаря

# Частотный словарь вредности
tokenizer_harm = Tokenizer(num_words=200, # объем словаря
                      filters='!"#$%&()*+,-–—/:;<=>?@[\\]^_`{|}~\t\n\xa0', # убираемые из текста ненужные символы 
                      lower=True, # приведение слов к нижнему регистру
                      split=' ', # разделитель слов
                      oov_token='unknown', # указание разделять по словам, а не по единичным символам
                      char_level=False # токен для слов, которые не вошли в словарь
                      )

# Частотный словарь диагнозов узких специалистов
tokenizer_Diag = Tokenizer(num_words=500, # объем словаря
                      filters='!"#$%&()*+,-–—./:;<=>?@[\\]^_`{|}~\t\n\xa0', # убираемые из текста ненужные символы 
                      lower=False, # приведение слов к нижнему регистру
                      split=' ', # разделитель слов
                      oov_token='unknown', # указание разделять по словам, а не по единичным символам
                      char_level=False # токен для слов, которые не вошли в словарь
                      )

# Частотный словарь рекомендаций узких специалистов
tokenizer_recom = Tokenizer(num_words=100, # объем словаря
                      filters='!"#$%&()*+,-–—./:;<=>?@[\\]^_`{|}~\t\n\xa0', # убираемые из текста ненужные символы 
                      lower=False, # приведение слов к нижнему регистру
                      split=' ', # разделитель слов
                      oov_token='unknown', # указание разделять по словам, а не по единичным символам
                      char_level=False # токен для слов, которые не вошли в словарь
                      )

# Вспомогательные функции для очистки строковых данных для приведения к удобномы для обратки виду
def purify(x):
    if isinstance(x, str):                # Если значение - строка:
        # Замена концов строк на пробелы, удаление символа с кодом 0xA0,
        # обрезка краевых пробелов и приведение к нижнему регистру
        x1 = re.sub(r',\s*(?=,|$)', '', x).strip(',')
    return x1

def clean_string(text): 
    # удаление знаков препинания
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text    

def clean_dict(dict_d):
  advanceddict = {}
  for key in dict_d:
    # проверяем, есть ли в ключе разделитель ","
    if re.search(',', key):
      # если есть, то разделяем ключ на список из отдельных ключей
      subkeys = key.split(',')
      # для каждого нового ключа создаем запись в новом словаре
      for subkey in subkeys:
        # копируем значение из старого словаря
        advanceddict[subkey] = dict_d[key]
    else:
      # если разделителя нет, то просто копируем ключ и значение
      advanceddict[key] = dict_d[key]

  return advanceddict

   

# Функция обработки столбца сводного диагноза
def diag_adv (df0):
  
  df1 = df0.copy()
  # Получим список всех сводных диагнозов 
  list_of_Diag = df1['Свод_Диагноз'].tolist()
  # Используя заготовленную функцию, очистим каждую строку от "мусора"
  clearlistdiag=[]
  for A in list_of_Diag:
    clearlistdiag.append(purify(A))
  # Поместим в новую колонку очищенные строки из полученного списка
  df1['Диагнозы'] = True
  df1['Диагнозы'] = clearlistdiag

  # Получаем новую таблицу данных
  df1=df1[[
      'Клиент',
      'КлиентДатаРождения',
      'ЗаключениеМК',
      'ГруппаЗдоровья',
      'Вредность',
      'Диагнозы',
      'Рекомендации']]
  return df1     

# Функция обработки столбца рекомендаций
def recom_adv (df1):
    df2 = df1.copy()
    # Получим список всех рекомендаций 
    list_of_recom = df2['Рекомендации'].tolist()
    # Используя заготовленную функцию, очистим каждую строку от "мусора"
    clearlistrecom=[]
    for A in list_of_recom:
      clearlistrecom.append(purify(A))
    # Поместим в новую колонку очищенные строки из полученного списка
    df2['Рекомендации'] = True
    df2['Рекомендации'] = clearlistrecom

    # Получаем новую таблицу данных
    df2=df2[[
        'Клиент',
        'КлиентДатаРождения',
        'ЗаключениеМК',
        'ГруппаЗдоровья',
        'Вредность',
        'Диагнозы',
        'Рекомендации'
        ]]
    return df2       

# Сформируем отдельную функцию, которая по передаваемой исходной таблицы создает сжатую
def getclin(df_in):
  # Получаем на вход некую таблицу данных вида аналогичного первоначально предоставленной
  # Подготовка таблицы данных
  # Убираем полностью пустые строки
  df_in = df_in.dropna(axis=1, how='all')

  # Убираем строки в которых отстутствует одно из встреченных заключений Профпатолога (ПП)
  df_in = df_in[df_in['ЗаключениеМК'].isin(['Годен',
                                      'ГоденСКоррекциейЗрения',
                                      'ВременноНегоден',
                                      'НуждаетсяВДообследованииИЛечении',
                                      'ОграниченноГоден',
                                      'ГоденБезРаботНаВысотах'])]

  # Приводим к 3-м классам (Годен, Негоден, ОграниченноГоден)
  df_in.loc[df_in['ЗаключениеМК'] == 'ВременноНегоден', 'ЗаключениеМК'] = 'Негоден'
  df_in.loc[df_in['ЗаключениеМК'] == 'НуждаетсяВДообследованииИЛечении', 'ЗаключениеМК'] = 'Негоден'
  df_in.loc[df_in['ЗаключениеМК'] == 'ГоденСКоррекциейЗрения', 'ЗаключениеМК'] = 'ОграниченноГоден'
  df_in.loc[df_in['ЗаключениеМК'] == 'ГоденБезРаботНаВысотах', 'ЗаключениеМК'] = 'ОграниченноГоден'

  # Для удобства обращения к столбцу, переименуем его в название из одного слова
  df_in.rename(columns = {'Вредные Факторы':'Вредность'}, inplace = True )
  df_in.rename(columns = {'Диспансерная Группа':'ГруппаЗдоровья'}, inplace = True )

  # Оставляем только значащие для нас колонки: Заключение ПП, Вредность и Диагнозы узких специалистов в текстовом представлении
  df_in = df_in[[
  'Клиент',
  'КлиентДатаРождения',

  'ЗаключениеМК',

  'ГруппаЗдоровья',
  'Вредность',

  'Психиатрия Наркология1 Диагноз Представление1',
  'Офтальмология1_ДиагнозПредставление1',
  'Оториноларингология1_ДиагнозПредставление1',
  'Гинекология1_ДиагнозПредставление1',
  'Стоматология1_ДиагнозПредставление1',
  'Дерматовенерология1_ДиагнозПредставление1',
  'Рентгенология1_ДиагнозПредставление1',
  'Неврология1_ДиагнозПредставление1',
  'Терапия1_ДиагнозПредставление1',
  'Хирургия1_ДиагнозПредставление1',
  'Офтальмология1_ДиагнозПредставление2',
  'Терапия2_ДиагнозПредставление1',

  'ЛабораторныеИсследования1_ХР_Рекомендация1Представление',
  'Рентгенология1_Заключение_ЗначениеПредставление',
  'ПсихиатрияНаркология1_ХР_Рекомендация1Представление',
  'Офтальмология1_ХР_Рекомендация1Представление',
  'Офтальмология1_ХР_Рекомендация2Представление',
  'Оториноларингология1_ХР_Рекомендация1Представление',
  'Гинекология1_ХР_Рекомендация1Представление',
  'Гинекология1_ХР_Рекомендация2Представление',
  'ФункциональнаяДиагностика1_ХР_Рекомендация1Представление',
  'Стоматология2_ХР_Рекомендация1Представление',
  'Терапия1_ХР_Рекомендация1Представление',
  'Терапия1_ХР_Рекомендация2Представление',
  'Дерматовенерология1_ХР_Рекомендация1Представление',
  'Дерматовенерология1_ХР_Рекомендация2Представление',
  'Неврология1_ХР_Рекомендация1Представление',
  'Хирургия1_ХР_Рекомендация1Представление',
  'Терапия2_ХР_Рекомендация1Представление',
  'Терапия2_ХР_Рекомендация2Представление'
  ]]

  # Заменяем отсутсвтующие записи узких специалистов в пробелы
  df_in=df_in.fillna('')

  # Обобщенный диагноз - собирем диагнозы узких специалистов в одну строку через запятую
  df_in['Свод_Диагноз'] = (df_in['Психиатрия Наркология1 Диагноз Представление1'].map(str) + ','
  + df_in['Офтальмология1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Оториноларингология1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Гинекология1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Стоматология1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Дерматовенерология1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Рентгенология1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Неврология1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Терапия1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Хирургия1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Офтальмология1_ДиагнозПредставление2'].map(str)+ ','
  + df_in['Терапия2_ДиагнозПредставление1'].map(str))



  # Обобщенная рекомендация
  df_in['Рекомендации'] = (df_in['ЛабораторныеИсследования1_ХР_Рекомендация1Представление'].map(str) + ','
  + df_in['Рентгенология1_Заключение_ЗначениеПредставление'].map(str) + ','
  + df_in['ПсихиатрияНаркология1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Офтальмология1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Офтальмология1_ХР_Рекомендация2Представление'].map(str)+ ','
  + df_in['Оториноларингология1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Гинекология1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Гинекология1_ХР_Рекомендация2Представление'].map(str)+ ','
  + df_in['ФункциональнаяДиагностика1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Стоматология2_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Терапия1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Терапия1_ХР_Рекомендация2Представление'].map(str)+ ','
  + df_in['Дерматовенерология1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Дерматовенерология1_ХР_Рекомендация2Представление'].map(str)+ ','
  + df_in['Неврология1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Хирургия1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Терапия2_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Терапия2_ХР_Рекомендация2Представление'].map(str))


  # Упрвостим таблицу данных до 7 столбцов
  df_in1 = df_in[[
      'Клиент',
      'КлиентДатаРождения',
      'ЗаключениеМК',
      'ГруппаЗдоровья',
      'Вредность',
      'Свод_Диагноз',
      'Рекомендации'
      ]]

  # Получаем новую таблицу данных
  df_res1=diag_adv(df_in1)
  df_res2=recom_adv(df_res1)

  # Удаляем строки с отстуствующими записями всех узких специалистов                    
  df_res = df_res2[df_res2['Диагнозы']!='']

  # обновляем номера индексов
  # df_res=df_res3.reset_index(drop=True)
  return df_res

# Функция преобразования данных из таблицы во входные данные для модели
def getXdata (df_res):
  # Извлечение текстов о вредности для выборки
  harm_text = extract_harm_text(df_res) 

  tokenizer_harm.fit_on_texts(harm_text)
  items_harm = list(tokenizer_harm.word_index.items())

  # Преобразование текстов в последовательность индексов согласно частотному словарю
  harm_seq = tokenizer_harm.texts_to_sequences(harm_text)

  # Преобразование последовательностей индексов в bag of words
  x_harm = tokenizer_harm.sequences_to_matrix(harm_seq)

  # Извлечение текстов о сводном диагнозе для выборки
  Diag_text = extract_Diag_text(df_res) 

  tokenizer_Diag.fit_on_texts(Diag_text)
  items_Diag = list(tokenizer_Diag.word_index.items())

  # Преобразование текстов в последовательность индексов согласно частотному словарю
  Diag_seq = tokenizer_Diag.texts_to_sequences(Diag_text)

  # Преобразование последовательностей индексов в bag of words
  x_diag = tokenizer_Diag.sequences_to_matrix(Diag_seq)


  # Извлечение текстов о рекомендациях для выборки
  recom_text = extract_recom_text(df_res) 

  tokenizer_recom.fit_on_texts(recom_text)
  items_recom = list(tokenizer_recom.word_index.items())

  # Преобразование текстов в последовательность индексов согласно частотному словарю
  recom_seq = tokenizer_recom.texts_to_sequences(recom_text)

  # Преобразование последовательностей индексов в bag of words
  x_recom = tokenizer_recom.sequences_to_matrix(recom_seq)

  # Получение данных о группе здоровья
  x_gz = extract_gz(df_res)
   

  return x_harm, x_diag, x_gz, x_recom


def predict(df_path='/content/drive/MyDrive/internsheep1//profpat_1.csv', model_path='content/drive/MyDrive/internsheep1/model_work.h5'):
    classes = {0: 'Негоден',
               1: 'ОграниченноГоден',
               2: 'Годен'}

    # Загружаем тестовую таблицу данных от Заказчика(здесь для примера загружается та же таблица)
    df_test = pd.read_csv('/content/drive/MyDrive/internsheep1//profpat_1.csv')

    # Обрабатываем таблицу данных до необходимого формата
    df_clin = getclin(df_test)

    # Вредность, диагнозы, рекомендации от УС
    x_test_harm, x_test_diag, x_test_gz, x_test_recom,  = getXdata (df_clin)

    # Значения реальных классов для контроля точности в формате класса и векторного представления класса
    y_data_test, y_vect_test = Y_to_OHE(df_clin)

    # Формируем списки всех входных и выходных данных
    y_vec_list = list(y_vect_test)
    y_data_list = list(y_data_test)
    client_list = list(df_clin['Клиент'])
    clientDB_list = list(df_clin['КлиентДатаРождения'])
    x_harm_list = list(df_clin['Вредность'])
    x_diag_list = list(df_clin['Диагнозы'])
    x_recom_list = list(df_clin['Рекомендации'])
    x_gz_list = list(df_clin['ГруппаЗдоровья'])           
    
    # Загружаем модель
    model = load_model(model_path)

    cls_df = np.argmax(model.predict([getXdata (df_clin)]), axis=-1)

    # y_class_list =list(classes[cls_df])
    y_class_list =list(cls_df)
    # Сформируем таблицу с выводом реальных значений и предсказания
    df_pred = pd.DataFrame({
                      'РеалКласс': y_data_list,
                      'Клиент': client_list,
                      'КлиентДатаРождения': clientDB_list,
                      'Заключение РС': y_class_list,
                      'ГруппаЗдоровья': x_gz_list,
                      'Вредный фактор': x_harm_list,
                      'Диагнозы': x_diag_list,
                      'Рекомендации': x_recom_list
                      })
    
    # Вывод полученных данных в рекомендательную таблицу
    df_pred.to_excel('pp_res.xlsx', index=False)    

In [5]:
predict(df_path='/content/drive/MyDrive/internsheep1/profpat_1.csv', model_path='/content/drive/MyDrive/internsheep1/model_18-90.h5')


<ipython-input-4-91ac4e17f1f8>:365: DtypeWarning: Columns (24,25,26,43,44,45,46,47,48,54,55,56,57,58,59,60,61,62,63,64,65,77,78,79,81,84,85,86,87,88,89,90,91,92,108,109,112,113,120,121,122,123,124,125,126,127,162,163,164,168,169,170,171,172,173,174,195,196,201,202,203,204,205,206,207,208,221,222,223,228,229,230,252,253,258,259,260,261,262,263,264,265,266,267,270,271,272,273,276,277,278,279,282,283,284,285,286,287,293,294,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,363,364,365,366,367,370,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,418,419,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,470,471,472,483,484,485,486,491,492,514,515,549,550,551,552,553,554,555,564,565,578,579,580,581,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600

45/45 [==============================] - 3s 44ms/step


In [7]:
# Сохранение кода скрипта в переменной listing1

listing1 = '''
# Работа с массивами данных
import numpy as np
from tensorflow.keras.models import load_model

# Регулярные выражения
import re

#Для работы с файлами 
import os 

# Модуль для работы с zip-архивами
from zipfile import ZipFile as Zip             

# Работа с табличными данными
import pandas as pd

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Токенизатор для преобразования текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Масштабирование данных
from sklearn.preprocessing import StandardScaler

!pip install tensorflow-addons

import tensorflow_addons as tfa

from tensorflow_addons.metrics import F1Score

# Задаем константы
# Количество классов
CLASS_COUNT = 3

# Задаем словарь классов
CLASS_DICT ={'Негоден':0, 'ОграниченноГоден':1, 'Годен':2}

# Задаем словарь групп здоровья
GZ_DICT ={'':0, 'I':1, 'II':2, 'IIIа':3, 'IIIб':4}

# Функция извлечения данных о вредных факторах

def extract_harm_text(df_harm):
    result = []
    COL_HARM   = df_harm.columns.get_loc('Вредность')
    # Для всех строк таблицы: собрать значения столбцов вредности
    for row in df_harm.values:
                    result.append(str(row[COL_HARM]).split(','))
    # Возврат в виде массива
    return result

# Функция извлечения данных о Диагнозе

def extract_Diag_text(df_diag):
    result = []
    COL_DIAG    = df_diag.columns.get_loc('Диагнозы')
    # Для всех строк таблицы: собрать значения сводного Диагноза
    for row in df_diag.values:
                    result.append(str(row[COL_DIAG]).split(','))
    # Возврат в виде массива
    return result

# Функция извлечения данных о Рекомендациях

def extract_recom_text(df_diag):
    result = []
    COL_RECOM    = df_diag.columns.get_loc('Рекомендации')
    # Для всех строк таблицы: собрать значения сводного Диагноза
    for row in df_diag.values:
                    result.append(str(row[COL_RECOM]).split(','))
    # Возврат в виде массива
    return result

# Функция извлечения данных о Группе здоровья OHE

def extract_gz(df):
  GZ_COUNT = 5
  gz_list_key = list(df.ГруппаЗдоровья.values)
  gz_list = []
  for GZ in gz_list_key:
    gz_list.append(GZ_DICT[GZ])
  
  gz_data = np.array(gz_list)            # Перевод общего списка меток класса в numpy-массив
  gz_res = utils.to_categorical(gz_data, GZ_COUNT)
  return gz_res

# Функция перевода классов в OHE
def Y_to_OHE(df):
  y_list_key = list(df.ЗаключениеМК.values)
  # print (y_list_key)
  y_list = []
  for CD in y_list_key:
    y_list.append(CLASS_DICT[CD])
  # print (y_list)

  y_data = np.array(y_list)            # Перевод общего списка меток класса в numpy-массив

  y_res = utils.to_categorical(y_data, CLASS_COUNT)
  return y_data, y_res

# Преобразование текстовых данных в числовые/векторные для обучения нейросетью
# Используется встроенный в Keras токенизатор для разбиения текста и построения частотного словаря

# Частотный словарь вредности
tokenizer_harm = Tokenizer(num_words=200, # объем словаря
                      filters='!"#$%&()*+,-–—/:;<=>?@[\\]^_`{|}~\t\n\xa0', # убираемые из текста ненужные символы 
                      lower=True, # приведение слов к нижнему регистру
                      split=' ', # разделитель слов
                      oov_token='unknown', # указание разделять по словам, а не по единичным символам
                      char_level=False # токен для слов, которые не вошли в словарь
                      )

# Частотный словарь диагнозов узких специалистов
tokenizer_Diag = Tokenizer(num_words=500, # объем словаря
                      filters='!"#$%&()*+,-–—./:;<=>?@[\\]^_`{|}~\t\n\xa0', # убираемые из текста ненужные символы 
                      lower=False, # приведение слов к нижнему регистру
                      split=' ', # разделитель слов
                      oov_token='unknown', # указание разделять по словам, а не по единичным символам
                      char_level=False # токен для слов, которые не вошли в словарь
                      )

# Частотный словарь рекомендаций узких специалистов
tokenizer_recom = Tokenizer(num_words=100, # объем словаря
                      filters='!"#$%&()*+,-–—./:;<=>?@[\\]^_`{|}~\t\n\xa0', # убираемые из текста ненужные символы 
                      lower=False, # приведение слов к нижнему регистру
                      split=' ', # разделитель слов
                      oov_token='unknown', # указание разделять по словам, а не по единичным символам
                      char_level=False # токен для слов, которые не вошли в словарь
                      )

# Вспомогательные функции для очистки строковых данных для приведения к удобномы для обратки виду
def purify(x):
    if isinstance(x, str):                # Если значение - строка:
        # Замена концов строк на пробелы, удаление символа с кодом 0xA0,
        # обрезка краевых пробелов и приведение к нижнему регистру
        x1 = re.sub(r',\s*(?=,|$)', '', x).strip(',')
    return x1

def clean_string(text): 
    # удаление знаков препинания
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text    

def clean_dict(dict_d):
  advanceddict = {}
  for key in dict_d:
    # проверяем, есть ли в ключе разделитель ","
    if re.search(',', key):
      # если есть, то разделяем ключ на список из отдельных ключей
      subkeys = key.split(',')
      # для каждого нового ключа создаем запись в новом словаре
      for subkey in subkeys:
        # копируем значение из старого словаря
        advanceddict[subkey] = dict_d[key]
    else:
      # если разделителя нет, то просто копируем ключ и значение
      advanceddict[key] = dict_d[key]

  return advanceddict

   

# Функция обработки столбца сводного диагноза
def diag_adv (df0):
  
  df1 = df0.copy()
  # Получим список всех сводных диагнозов 
  list_of_Diag = df1['Свод_Диагноз'].tolist()
  # Используя заготовленную функцию, очистим каждую строку от "мусора"
  clearlistdiag=[]
  for A in list_of_Diag:
    clearlistdiag.append(purify(A))
  # Поместим в новую колонку очищенные строки из полученного списка
  df1['Диагнозы'] = True
  df1['Диагнозы'] = clearlistdiag

  # Получаем новую таблицу данных
  df1=df1[[
      'Клиент',
      'КлиентДатаРождения',
      'ЗаключениеМК',
      'ГруппаЗдоровья',
      'Вредность',
      'Диагнозы',
      'Рекомендации']]
  return df1     

# Функция обработки столбца рекомендаций
def recom_adv (df1):
    df2 = df1.copy()
    # Получим список всех рекомендаций 
    list_of_recom = df2['Рекомендации'].tolist()
    # Используя заготовленную функцию, очистим каждую строку от "мусора"
    clearlistrecom=[]
    for A in list_of_recom:
      clearlistrecom.append(purify(A))
    # Поместим в новую колонку очищенные строки из полученного списка
    df2['Рекомендации'] = True
    df2['Рекомендации'] = clearlistrecom

    # Получаем новую таблицу данных
    df2=df2[[
        'Клиент',
        'КлиентДатаРождения',
        'ЗаключениеМК',
        'ГруппаЗдоровья',
        'Вредность',
        'Диагнозы',
        'Рекомендации'
        ]]
    return df2       

# Сформируем отдельную функцию, которая по передаваемой исходной таблицы создает сжатую
def getclin(df_in):
  # Получаем на вход некую таблицу данных вида аналогичного первоначально предоставленной
  # Подготовка таблицы данных
  # Убираем полностью пустые строки
  df_in = df_in.dropna(axis=1, how='all')

  # Убираем строки в которых отстутствует одно из встреченных заключений Профпатолога (ПП)
  df_in = df_in[df_in['ЗаключениеМК'].isin(['Годен',
                                      'ГоденСКоррекциейЗрения',
                                      'ВременноНегоден',
                                      'НуждаетсяВДообследованииИЛечении',
                                      'ОграниченноГоден',
                                      'ГоденБезРаботНаВысотах'])]

  # Приводим к 3-м классам (Годен, Негоден, ОграниченноГоден)
  df_in.loc[df_in['ЗаключениеМК'] == 'ВременноНегоден', 'ЗаключениеМК'] = 'Негоден'
  df_in.loc[df_in['ЗаключениеМК'] == 'НуждаетсяВДообследованииИЛечении', 'ЗаключениеМК'] = 'Негоден'
  df_in.loc[df_in['ЗаключениеМК'] == 'ГоденСКоррекциейЗрения', 'ЗаключениеМК'] = 'ОграниченноГоден'
  df_in.loc[df_in['ЗаключениеМК'] == 'ГоденБезРаботНаВысотах', 'ЗаключениеМК'] = 'ОграниченноГоден'

  # Для удобства обращения к столбцу, переименуем его в название из одного слова
  df_in.rename(columns = {'Вредные Факторы':'Вредность'}, inplace = True )
  df_in.rename(columns = {'Диспансерная Группа':'ГруппаЗдоровья'}, inplace = True )

  # Оставляем только значащие для нас колонки: Заключение ПП, Вредность и Диагнозы узких специалистов в текстовом представлении
  df_in = df_in[[
  'Клиент',
  'КлиентДатаРождения',

  'ЗаключениеМК',

  'ГруппаЗдоровья',
  'Вредность',

  'Психиатрия Наркология1 Диагноз Представление1',
  'Офтальмология1_ДиагнозПредставление1',
  'Оториноларингология1_ДиагнозПредставление1',
  'Гинекология1_ДиагнозПредставление1',
  'Стоматология1_ДиагнозПредставление1',
  'Дерматовенерология1_ДиагнозПредставление1',
  'Рентгенология1_ДиагнозПредставление1',
  'Неврология1_ДиагнозПредставление1',
  'Терапия1_ДиагнозПредставление1',
  'Хирургия1_ДиагнозПредставление1',
  'Офтальмология1_ДиагнозПредставление2',
  'Терапия2_ДиагнозПредставление1',

  'ЛабораторныеИсследования1_ХР_Рекомендация1Представление',
  'Рентгенология1_Заключение_ЗначениеПредставление',
  'ПсихиатрияНаркология1_ХР_Рекомендация1Представление',
  'Офтальмология1_ХР_Рекомендация1Представление',
  'Офтальмология1_ХР_Рекомендация2Представление',
  'Оториноларингология1_ХР_Рекомендация1Представление',
  'Гинекология1_ХР_Рекомендация1Представление',
  'Гинекология1_ХР_Рекомендация2Представление',
  'ФункциональнаяДиагностика1_ХР_Рекомендация1Представление',
  'Стоматология2_ХР_Рекомендация1Представление',
  'Терапия1_ХР_Рекомендация1Представление',
  'Терапия1_ХР_Рекомендация2Представление',
  'Дерматовенерология1_ХР_Рекомендация1Представление',
  'Дерматовенерология1_ХР_Рекомендация2Представление',
  'Неврология1_ХР_Рекомендация1Представление',
  'Хирургия1_ХР_Рекомендация1Представление',
  'Терапия2_ХР_Рекомендация1Представление',
  'Терапия2_ХР_Рекомендация2Представление'
  ]]

  # Заменяем отсутсвтующие записи узких специалистов в пробелы
  df_in=df_in.fillna('')

  # Обобщенный диагноз - собирем диагнозы узких специалистов в одну строку через запятую
  df_in['Свод_Диагноз'] = (df_in['Психиатрия Наркология1 Диагноз Представление1'].map(str) + ','
  + df_in['Офтальмология1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Оториноларингология1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Гинекология1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Стоматология1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Дерматовенерология1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Рентгенология1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Неврология1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Терапия1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Хирургия1_ДиагнозПредставление1'].map(str)+ ','
  + df_in['Офтальмология1_ДиагнозПредставление2'].map(str)+ ','
  + df_in['Терапия2_ДиагнозПредставление1'].map(str))



  # Обобщенная рекомендация
  df_in['Рекомендации'] = (df_in['ЛабораторныеИсследования1_ХР_Рекомендация1Представление'].map(str) + ','
  + df_in['Рентгенология1_Заключение_ЗначениеПредставление'].map(str) + ','
  + df_in['ПсихиатрияНаркология1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Офтальмология1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Офтальмология1_ХР_Рекомендация2Представление'].map(str)+ ','
  + df_in['Оториноларингология1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Гинекология1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Гинекология1_ХР_Рекомендация2Представление'].map(str)+ ','
  + df_in['ФункциональнаяДиагностика1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Стоматология2_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Терапия1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Терапия1_ХР_Рекомендация2Представление'].map(str)+ ','
  + df_in['Дерматовенерология1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Дерматовенерология1_ХР_Рекомендация2Представление'].map(str)+ ','
  + df_in['Неврология1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Хирургия1_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Терапия2_ХР_Рекомендация1Представление'].map(str)+ ','
  + df_in['Терапия2_ХР_Рекомендация2Представление'].map(str))


  # Упрвостим таблицу данных до 7 столбцов
  df_in1 = df_in[[
      'Клиент',
      'КлиентДатаРождения',
      'ЗаключениеМК',
      'ГруппаЗдоровья',
      'Вредность',
      'Свод_Диагноз',
      'Рекомендации'
      ]]

  # Получаем новую таблицу данных
  df_res1=diag_adv(df_in1)
  df_res2=recom_adv(df_res1)

  # Удаляем строки с отстуствующими записями всех узких специалистов                    
  df_res = df_res2[df_res2['Диагнозы']!='']

  # обновляем номера индексов
  # df_res=df_res3.reset_index(drop=True)
  return df_res

# Функция преобразования данных из таблицы во входные данные для модели
def getXdata (df_res):
  # Извлечение текстов о вредности для выборки
  harm_text = extract_harm_text(df_res) 

  tokenizer_harm.fit_on_texts(harm_text)
  items_harm = list(tokenizer_harm.word_index.items())

  # Преобразование текстов в последовательность индексов согласно частотному словарю
  harm_seq = tokenizer_harm.texts_to_sequences(harm_text)

  # Преобразование последовательностей индексов в bag of words
  x_harm = tokenizer_harm.sequences_to_matrix(harm_seq)

  # Извлечение текстов о сводном диагнозе для выборки
  Diag_text = extract_Diag_text(df_res) 

  tokenizer_Diag.fit_on_texts(Diag_text)
  items_Diag = list(tokenizer_Diag.word_index.items())

  # Преобразование текстов в последовательность индексов согласно частотному словарю
  Diag_seq = tokenizer_Diag.texts_to_sequences(Diag_text)

  # Преобразование последовательностей индексов в bag of words
  x_diag = tokenizer_Diag.sequences_to_matrix(Diag_seq)


  # Извлечение текстов о рекомендациях для выборки
  recom_text = extract_recom_text(df_res) 

  tokenizer_recom.fit_on_texts(recom_text)
  items_recom = list(tokenizer_recom.word_index.items())

  # Преобразование текстов в последовательность индексов согласно частотному словарю
  recom_seq = tokenizer_recom.texts_to_sequences(recom_text)

  # Преобразование последовательностей индексов в bag of words
  x_recom = tokenizer_recom.sequences_to_matrix(recom_seq)

  # Получение данных о группе здоровья
  x_gz = extract_gz(df_res)
   

  return x_harm, x_diag, x_gz, x_recom


def predict(df_path='/content/drive/MyDrive/internsheep1//profpat_1.csv', model_path='content/drive/MyDrive/internsheep1/model_work.h5'):
    classes = {0: 'Негоден',
               1: 'ОграниченноГоден',
               2: 'Годен'}

    # Загружаем тестовую таблицу данных от Заказчика(здесь для примера загружается та же таблица)
    df_test = pd.read_csv('/content/drive/MyDrive/internsheep1//profpat_1.csv')

    # Обрабатываем таблицу данных до необходимого формата
    df_clin = getclin(df_test)

    # Вредность, диагнозы, рекомендации от УС
    x_test_harm, x_test_diag, x_test_gz, x_test_recom,  = getXdata (df_clin)

    # Значения реальных классов для контроля точности в формате класса и векторного представления класса
    y_data_test, y_vect_test = Y_to_OHE(df_clin)

    # Формируем списки всех входных и выходных данных
    y_vec_list = list(y_vect_test)
    y_data_list = list(y_data_test)
    client_list = list(df_clin['Клиент'])
    clientDB_list = list(df_clin['КлиентДатаРождения'])
    x_harm_list = list(df_clin['Вредность'])
    x_diag_list = list(df_clin['Диагнозы'])
    x_recom_list = list(df_clin['Рекомендации'])
    x_gz_list = list(df_clin['ГруппаЗдоровья'])           
    
    # Загружаем модель
    model = load_model(model_path)

    cls_df = np.argmax(model.predict([getXdata (df_clin)]), axis=-1)

    # y_class_list =list(classes[cls_df])
    y_class_list =list(cls_df)
    # Сформируем таблицу с выводом реальных значений и предсказания
    df_pred = pd.DataFrame({
                      'РеалКласс': y_data_list,
                      'Клиент': client_list,
                      'КлиентДатаРождения': clientDB_list,
                      'Заключение РС': y_class_list,
                      'ГруппаЗдоровья': x_gz_list,
                      'Вредный фактор': x_harm_list,
                      'Диагнозы': x_diag_list,
                      'Рекомендации': x_recom_list
                      })
    
    # Вывод полученных данных в рекомендательную таблицу
    df_pred.to_excel('pp_res.xlsx', index=False)    
    
'''

# Запись содержимого переменной в файл скрипта

with open('script.py', 'w') as f:  # Создание / открытие файла 
    f.write(listing1)             # Запись в файл значения переменной text_code

In [8]:
import tensorflow as tf

# Загрузка модели
model_download = tf.keras.models.load_model('/content/drive/MyDrive/internsheep1/model_18-90.h5')

In [10]:
# Библиотека для работы с файлами
from shutil import copyfile

copyfile(f'/content/drive/MyDrive/internsheep1/profpat_1.csv', f'/content/profpat_1.csv')
copyfile(f'/content/drive/MyDrive/internsheep1/model_18-90.h5', f'/content/model_18-90.h5')

'/content/model_18-90.h5'

In [11]:
label_txt = '''

Таблица на 1980 записей пациентов

'''

with open('label.txt', 'w') as f:  # Создание / открытие файла 
    f.write(label_txt.strip())     # Запись в файл значения переменной

In [12]:
import os
from zipfile import ZipFile as Zip             # Модуль для работы с zip-архивами

# Список всех файлов
integrate_files = ['model_18-90.h5', 'script.py', 'label.txt', 'profpat_1.csv']

zipArchive = Zip('/content/archive.zip', 'w')  # Открытие файла, если такого не существует - будет создан новый

for file in integrate_files:                   # Для всех всем нужных файлов:
    if os.path.exists(file):                   # Если файл существует,
        zipArchive.write(file)                 # то добавление его в архив
    else:
        print(f'Файл {file} отсутствует!')     # иначе вывод на экран названия отсутствующего файла

zipArchive.close()                             # В финале архив следует закрыть, как и обычный файл

In [13]:
copyfile(f'/content/archive.zip', f'/content/drive/MyDrive/internsheep1/archive.zip')

'/content/drive/MyDrive/internsheep1/archive.zip'